<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ моделей</a></span></li></ul></div>

# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [1]:
#загружаем библиотеки
import pandas as pd
import numpy as np
import requests
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

## Подготовка данных

Первым делом подготовим данные. Загрузим их и посмотрим на пропуски или аномалии. Далее постараемся их исправить.

In [2]:
try:
    data = pd.read_csv('autos.csv')
except:
    data = pd.read_csv('/datasets/autos.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


In [4]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [5]:
data['Price'].describe()

count    354369.000000
mean       4416.656776
std        4514.158514
min           0.000000
25%        1050.000000
50%        2700.000000
75%        6400.000000
max       20000.000000
Name: Price, dtype: float64

Мы видим, что в целевом признаке есть пропуски. Т.к. наша цель — научиться предсказывать стоимость, строчки с пропусками в соответствующем столбце вряд ли будут нам полезны. Можно их удалить. Крайние значения, вроде 1, оставим, поскольку точно не знаем их происхождения и в то же время хотим, чтобы наша модель умела работать с данными, которые встречаются в реальной жизни.

In [6]:
data = data.drop(data[data['Price'] == 0].index)
deleted = (1 - len(data) / 354369)
print('Всего удалено данных: {:.2%}'.format(deleted))

Всего удалено данных: 3.04%


В данных присутствуют столбцы разных типов — здесь и категориальные, и численные данные. Также присутствуют даты, но в формате object. Два столбца — с датой скачивания объявления и с датой последнего просмотра — можно объединить в один: сколько дней объявление не просматривали до скачивания. Гипотетически, этот фактор может влиять на стоимость. Например, чем дольше объявление не просматривают, тем вероятнее что его цена высока. 

In [7]:
data['LastSeen'] = pd.to_datetime(data['LastSeen'], format='%Y-%m-%d %H:%M:%S') #переведем столбцы в специальный формат
data['DateCrawled'] = pd.to_datetime(data['DateCrawled'], format='%Y-%m-%d %H:%M:%S')

data['TimeNoSee'] = data['LastSeen'].dt.date - data['DateCrawled'].dt.date #вычтем один из другого
data['TimeNoSee'] = data['TimeNoSee'].apply(lambda x: int(str(x)[0:2])) #достанем числовое значение

In [8]:
data['Price'].corr(data['TimeNoSee'])

0.1491966187422049

Корреляция хоть и совсем маленькая, но прослеживается. Теперь можем удалить все столбцы с датами, включая день создания объявления — дальше они нам вряд ли понадобятся.

In [9]:
data = data.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1)

Посмотрим, на столбец с годом регистрации авто и на его значения.

In [10]:
data["RegistrationYear"].describe()

count    343597.000000
mean       2004.089797
std          78.413225
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: RegistrationYear, dtype: float64

In [11]:
len(data[(data["RegistrationYear"] < 1885) | (data["RegistrationYear"] > 2016)])

13832

Как видим, в нём присутствуют аномальные значения, вроде 1000 и 9999. Заменим на 0 все значения меньше 1885 года (год появления авто) и выше 2016 (все объявления идут от этого года), поскольку их слишком много, чтобы удалять, а восстановить их достоверно мы не сможем.

In [12]:
data["RegistrationYear"].where((data["RegistrationYear"] > 1885) & (data["RegistrationYear"] < 2016), 0, inplace=True)

Теперь перейдем к категориальным признакам и попробуем заменить их модой, найденной по набору связанных факторов. Например, в определении типа кузова нам поможет модель, год регистрации, коробка передач и некоторые другие признаки. Конечно, такое заполнение не дает 100% гарантию правильного ответа, но вероятность угадать будет точно выше 50%.

In [13]:
data[(data['Model'] == 'golf') & (data['RegistrationYear'] == 2000) & (data['Gearbox'] == 'manual') &
                (data['FuelType'] == 'petrol') & (data['Power'] == 75)].groupby('VehicleType')['VehicleType'].count()

VehicleType
convertible     14
coupe            1
other            3
sedan          276
small           72
wagon           53
Name: VehicleType, dtype: int64

Например, авто с вышеуказанными признаками с вероятностью в 65% окажется седаном. Если похожих авто не найдется, поставим категорию 'unknown'.

In [14]:
def find_mode(x):
    try:
        return x.mode().iloc[0]
    except:
        return 'unknown'

In [15]:
%%time
for column in ['VehicleType', 'Gearbox', 'FuelType']:
    sect_cols = ['Model', 'VehicleType', 'Gearbox', 'FuelType', 'Power', 'RegistrationYear']
    sect_cols.remove(column)
    section = data[data[sect_cols[0]].notna() & data[sect_cols[1]].notna() & data[sect_cols[2]].notna() &
                                data[sect_cols[3]].notna() & data[sect_cols[4]].notna()]
    filler = section.groupby(sect_cols)[column].apply(lambda x: x.fillna(find_mode(x)))
    data[column].fillna(filler, inplace=True)

CPU times: user 2min 29s, sys: 2.1 s, total: 2min 32s
Wall time: 2min 33s


In [16]:
%%time
sect_cols = ['Brand', 'VehicleType', 'Gearbox', 'FuelType', 'Power', 'RegistrationYear']
section = data[data[sect_cols[0]].notna() & data[sect_cols[1]].notna() & data[sect_cols[2]].notna() &
                                data[sect_cols[3]].notna() & data[sect_cols[4]].notna() & data[sect_cols[5]].notna()]
mod_filler = section.groupby(sect_cols)[column].apply(lambda x: x.fillna(find_mode(x)))
data['Model'].fillna(mod_filler, inplace=True)

CPU times: user 54.4 s, sys: 579 ms, total: 54.9 s
Wall time: 56 s


Теперь удалим все строки, в которых пропущены сразу все рассмотренные нами значения.

In [17]:
data = data.drop(data[data['Gearbox'].isna() & data['VehicleType'].isna() &
                                                          data['FuelType'].isna() & data['Model'].isna()].index)
deleted += (1 - len(data) / 354369)
print('Всего удалено данных: {:.2%}'.format(deleted))

Всего удалено данных: 6.54%


Оставшиеся пропуски в этих столбцах заменим на 'unknown'. Также этим значением заменим пропуски в столбце 'NotRepaired', поскольку никак иначе мы его восстановить не можем.

In [18]:
for column in ['VehicleType', 'Gearbox', 'FuelType', 'Model', 'NotRepaired']:
    data[column].fillna('unknown', inplace=True)

Перейдем к столбцу с почтовыми индексами.

In [19]:
print('В нём {} уникальных значений'.format(len(data['PostalCode'].unique())))

В нём 8135 уникальных значений


Т.к. географическое положение может влиять на стоимость авто, нам важно их сохранить, но, вероятно, лучше сократить кол-во уникальных значений, поскольку признак категориальный. Можно воспользоваться сторонним источником и заменить значения индексов на города. Загрузим справочник индексов. Он уже подготовлен.

In [20]:
try:
    postcodes = pd.read_csv('de_postal_codes.csv')
except:
    postcodes = pd.read_csv('https://raw.githubusercontent.com/julialysova/Postal-codes/main/de_postal_codes.csv')
postcodes.head()

,PostalCode,PlaceName,State,State Abbreviation,City,Latitude,Longitude,Unnamed: 7
0,1945,Kroppen,Brandenburg,BB,Oberspreewald-Lausitz,51.3833,13.8000,NaN
1,1968,Schipkau,Brandenburg,BB,Oberspreewald-Lausitz,51.5456,13.9121,NaN
2,1979,Lauchhammer,Brandenburg,BB,Oberspreewald-Lausitz,51.5000,13.7667,NaN
3,1983,GroЯrдschen,Brandenburg,BB,Oberspreewald-Lausitz,51.5833,14.0000,NaN
4,1987,Schwarzheide,Brandenburg,BB,Oberspreewald-Lausitz,51.4653,13.8680,NaN


In [21]:
postcodes['PostalCode'] = postcodes['PostalCode'].astype('int64') #переведем индексы в числовое значение
postcodes.set_index('PostalCode', inplace=True) #для простого поиска заменим указатель на столбец индексов

In [22]:
def find_city(postal, table):
    try:
        return table.loc[postal, 'City']
    except:
        return 'unknown'

In [23]:
data['City'] = data['PostalCode'].apply(find_city, table=postcodes) #заменяем все на города
data = data.drop(['PostalCode'], axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341981 entries, 0 to 354368
Data columns (total 14 columns):
Price                341981 non-null int64
VehicleType          341981 non-null object
RegistrationYear     341981 non-null int64
Gearbox              341981 non-null object
Power                341981 non-null int64
Model                341981 non-null object
Kilometer            341981 non-null int64
RegistrationMonth    341981 non-null int64
FuelType             341981 non-null object
Brand                341981 non-null object
NotRepaired          341981 non-null object
NumberOfPictures     341981 non-null int64
TimeNoSee            341981 non-null int64
City                 341981 non-null object
dtypes: int64(7), object(7)
memory usage: 39.1+ MB


In [24]:
len(data['City'].unique())

421

Теперь у нас 421 уникальное значение в столбце, что проще предсказать.

В данном шаге мы произвели обработку данных, заполнили пропуски и преобразовали некоторые признаки для удобства дальнейшей работы. В следующем шаге мы преобразуем данные так, чтобы на них можно было обучить модель, и попробуем поработать с разными моделями.

## Обучение моделей

Перейдем к выделению различных групп признаков. У нас есть категориальные, которые мы преобразуем по методу OHE; порядковые — Ordinal Encoding и численные, которые мы будем маштабировать. Модель и город мы отнесем к порядковым, поскольку они слишком увеличат размерность, если применить к ним OHE, и моделям будет сложно справиться с данными.

In [25]:
categorical = ['VehicleType', 'Gearbox', 'FuelType', 'Brand', 'NotRepaired']
numerical = ['Power', 'Kilometer', 'NumberOfPictures', 'TimeNoSee']
ordinal = ['Model', 'City', 'RegistrationYear']

Разделим данные на фичи и целевой признак.

In [26]:
target = data['Price']
features = data.drop(['Price'], axis=1)

In [27]:
target.describe()

count    341981.000000
mean       4568.749469
std        4517.229384
min           1.000000
25%        1200.000000
50%        2900.000000
75%        6500.000000
max       20000.000000
Name: Price, dtype: float64

Произведем преобразование категориальных признаков.

In [28]:
features_categorical = pd.get_dummies(features[categorical], drop_first=True)
features[features_categorical.columns] = features_categorical
features = features.drop(categorical, axis=1)

Далее преобразуем порядковые.

In [29]:
encoder = OrdinalEncoder()
encoder.fit(features[ordinal])
features[ordinal] = encoder.transform(features[ordinal])

Разделим данные на обучающую и тестовую выборки.

In [30]:
features_train, features_test, target_train, target_test = train_test_split(
                                                features, target, test_size=0.25, random_state=12345)

print('Размеры обучающей выборки:', features_train.shape)
print('Размеры валидационной выборки:', features_test.shape)

Размеры обучающей выборки: (256485, 66)
Размеры валидационной выборки: (85496, 66)


Отмасштабируем признаки для обучающей и тестовой выборки отдельно.

In [31]:
scaler = StandardScaler()
scaler.fit(features_train)

features_train_scaled = scaler.transform(features_train)
features_test_scaled = scaler.transform(features_test)

Перейдем к обучению моделей. Для проверки их качества мы будем использовать метрику RMSE, для вычисления которой создадим специальную функцию.

In [32]:
def rmse(pred, target):
    return np.sqrt(mean_squared_error(pred, target))

RMSE_scorer = make_scorer(rmse, greater_is_better=False)

Начнем обучение с решающего дерева и проверим, как быстро и как качественно он справляется с нашими данными.

In [51]:
tree_parameters = {'max_depth':range(1,20,2)}
tree = DecisionTreeRegressor(random_state=12345)
tree_grid = GridSearchCV(tree, tree_parameters, cv=5, verbose=1, iid=False, scoring=RMSE_scorer)

tree_best = tree_grid.fit(features_train_scaled, target_train)
tree_predictions = tree_best.predict(features_train_scaled)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  2.4min finished


In [52]:
tree_rmse = rmse(target_train, tree_predictions)
tree_rmse

1661.7670449349407

Ошибка на обучающих данных составила 1661,767, а время работы модели — 2,5 минуты на 50 обучений. Проверим результат модели на тестовых данных.

In [35]:
%%time
tree_predictions_test = tree_best.predict(features_test_scaled)
tree_rmse_test = rmse(target_test, tree_predictions_test)
tree_rmse_test

CPU times: user 35.9 ms, sys: 101 µs, total: 36 ms
Wall time: 34.3 ms


1949.9240271824424

Присутствует переобучение, но ошибка не превышает 2 тыс. Однако такой показатель всё еще кажется большим, поскольку превышает стоимость 25% автомобилей в нашей подборке. Теперь посмотрим на случайный лес.

In [36]:
forest_parameters = {'max_depth':range(1,11,2), 'n_estimators':range(1,40,10), 'max_leaf_nodes':[10,50]}
forest = RandomForestRegressor(random_state=12345)
forest_grid = GridSearchCV(forest, forest_parameters, scoring=RMSE_scorer, cv=2, verbose=1, iid=False)

forest_best = forest_grid.fit(features_train_scaled, target_train)
forest_predictions = forest_best.predict(features_train_scaled)

Fitting 2 folds for each of 40 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed: 10.0min finished


In [37]:
forest_rmse = rmse(target_train, tree_predictions)
forest_rmse

1661.7670449349407

In [38]:
%%time
forest_predictions_test = forest_best.predict(features_test_scaled)
forest_rmse_test = rmse(target_test, forest_predictions_test)
forest_rmse_test

CPU times: user 118 ms, sys: 34 µs, total: 118 ms
Wall time: 121 ms


2314.2103890417907

Случайный лес дал такой же показатель на обучающих данных, что и решающее дерево, но переобучение сильнее, поскольку на тестовых данных ошибка составила 2314,21. Время для 80 обучений — 9,7 мин.

Теперь посмотрим, как с данными справится бустинг LightGBM. Разделим и преобразуем заново, так как для категориальных признаков мы будем использовать встроенную в библиотеку lgb функцию работы с ними. Для этого их всё же придется преобразовать в численные (порядковые)

In [39]:
features_lgb = data.drop('Price', axis=1)

In [40]:
encoder.fit(features_lgb[categorical + ordinal])
features_lgb[categorical + ordinal] = encoder.transform(features_lgb[categorical + ordinal])

In [41]:
features_train_lgb, features_test_lgb, target_train_lgb, target_test_lgb = train_test_split(
                                                features_lgb, target, test_size=0.25, random_state=12345)

print('Размеры обучающей выборки:', features_train_lgb.shape)
print('Размеры валидационной выборки:', features_test_lgb.shape)

Размеры обучающей выборки: (256485, 13)
Размеры валидационной выборки: (85496, 13)


In [42]:
lgbm_parameters = {'num_leaves': [30, 100], 'learning_rate': [0.1, 0.03], 'max_depth': [10, 50]}
cat = categorical + ['Model','City']

In [43]:
%%time
lgbm_reg = lgb.LGBMRegressor(random_state=12345)
lgbm_grid = GridSearchCV(lgbm_reg, lgbm_parameters, scoring=RMSE_scorer, cv=2, verbose=1, iid=False)

lgbm_trained = lgbm_grid.fit(features_train_lgb, target_train_lgb, categorical_feature=cat)

lgbm_trained

Fitting 2 folds for each of 8 candidates, totalling 16 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Brand', 'City', 'FuelType', 'Gearbox', 'Model', 'NotRepaired', 'VehicleType']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Brand', 'City', 'FuelType', 'Gearbox', 'Model', 'NotRepaired', 'VehicleType']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Brand', 'City', 'FuelType', 'Gearbox', 'Model', 'NotRepaired', 'VehicleType']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/

CPU times: user 13min 1s, sys: 4.15 s, total: 13min 5s
Wall time: 13min 13s


GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=12345,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid=False, n_jobs=None,
             param_grid={'learning_rate': [0.1, 0.03], 'max_depth': [10, 50],
                         'num_leaves': [30, 100]},
             pre_dispatch='2*n_jobs', ref

In [44]:
lgbm_predictions = lgbm_trained.predict(features_train_lgb)
lgbm_rmse = rmse(target_train_lgb, lgbm_predictions)
lgbm_rmse

1364.3102970490654

In [45]:
%%time
lgbm_predictions_test = lgbm_trained.predict(features_test_lgb)

lgbm_rmse_test = rmse(target_test_lgb, lgbm_predictions_test)
lgbm_rmse_test

CPU times: user 2.17 s, sys: 0 ns, total: 2.17 s
Wall time: 2.19 s


1605.813593869075

Полученный результат как на обучении, так и на тесте лучше, чем у предыдущих моделей (1364,31 и 1605,81 соответственно). Время на 16 итераций - 8 мин 30 сек.

На этом шаге мы обучили три разные модели, получили результаты и время обучения каждой модели. В следующем шаге проанализируем полученные результаты.

## Анализ моделей

Мы провели обучение 3 регрессионных моделей: решающего дерева, случайного леса и градиентного бустинга. 
Самым быстрым было решающее дерево — 3 секунды на 1 итерацию. Результат на тестовой выборке не превысил 2 тыс.
Решающему лесу для одного обучения понадобилось 7,3 секунд, при этом ошибка на тестовой выборке превысила 2,3 тыс.
Самая маленькая ошибка (1,6 тыс.) была получена с помощью градиентного бустинга. При этом он был самым медленным — на одну итерацию ушло 30 секунд. Самое быстрое предсказание также было сделано с помощью решающего дерева.
Если ставить на весы и скорость и качество предсказания, то лучше обратиться к решающему дереву — оно делает относительно стабильные предсказания и делает это быстрее остальных испытанных моделей. Несмотря на то, что градиентный бустинг показывает наименьшую ошибку, он не подходит к запросу нашего клиента, которому нужно давать быстрые предсказания пользователям сервиса.